## Using custom python function as guard rail validator

### Define function

In [2]:
def my_validator(data: str) -> bool:
    return len(data) <= 10

### Using guard directly

In [3]:
from evidently.guardrails import PythonFunction

def process(input: str) -> str:
    PythonFunction(my_validator).validate(input)
    return input

In [4]:
process("My text")

'My text'

In [12]:
from evidently.guardrails import GuardException
# try / catch is to prevent jupyter to fail cell

try:
    process("My tooooo long text")
except GuardException as e:
    import traceback
    traceback.print_exception(e)

Traceback (most recent call last):
  File "C:\Users\v.morov\AppData\Local\Temp\ipykernel_49196\2234650047.py", line 4, in <module>
    process("My tooooo long text")
    ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\v.morov\AppData\Local\Temp\ipykernel_49196\1741774715.py", line 2, in process
    PythonFunction(my_validator).validate(input)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^
  File "C:\Users\v.morov\projects\evidently\src\evidently\guardrails\guards.py", line 25, in validate
    raise GuardException(f"{self.function.__name__} validation failed")
evidently.guardrails.core.GuardException: my_validator validation failed


### Using guard with decorator

In [ ]:
from evidently.guardrails import guard
from evidently.guardrails import PythonFunction

In [ ]:
@guard(PythonFunction(my_validator), input_arg="data")
def process_with_decorator(data: str) -> str:
    return data

In [ ]:
@guard([PythonFunction(my_validator), PythonFunction(my_validator)], input_arg="data")
def process_with_decorator_and_multiple_guards(data: str) -> str:
    return data

In [ ]:
from evidently.guardrails import GuardException
# try / catch is to prevent jupyter to fail cel

try:
    process_with_decorator("My tooooo long text")
except GuardException as e:
    import traceback
    traceback.print_exception(e)

### Handle failed guard in code

In [ ]:
from evidently.guardrails import GuardException
from evidently.guardrails import PythonFunction

def process_with_handle(input: str) -> str:
    try:
        PythonFunction(my_validator).validate(input)
        return input
    except GuardException as e:
        return f"Validation failed: {e}"

In [6]:
process_with_handle("My tooooo long text")

'Validation failed: my_validator validation failed'

## Examples of Guards

### PII Check

In [6]:
from evidently.guardrails import PIICheck


PIICheck().validate("My address is Some city")


{'category': 'PII', 'reasoning': "The text contains a person's address, which is considered personally identifiable information (PII) as it can be used to identify an individual."}


In [ ]:
@guard(PIICheck())
def process_with_pii(input: str) -> str:
    return input

### Includes Words

In [5]:
from evidently.guardrails import IncludesWords

IncludesWords(["word"], mode="all", lemmatize=True).validate("some words in text")

In [ ]:
@guard(IncludesWords(["word"]))
def process_with_includes(input: str) -> str:
    return input